In [11]:
from opentrons import simulate, execute, protocol_api
import pandas as pd
import numpy as np
import json
import os
import glob

In [12]:
# protocol_api.labware.get_all_labware_definitions()

In [3]:
def custom_labware_dict(labware_dir_path):
    """Given the path of a folder of custom labware .json files will create dict
    of key = name and value = labware definition to be loaded using protocol.load_labware_from_definition 
    versus typical protocol.load_labware"""
    os.chdir(labware_dir_path)
    labware_dict = {}
    for file in glob.glob("*.json"):
        with open(file) as labware_file:
            labware_name = os.path.splitext(file)[0] # removes the .json extnesion
            labware_def = json.load(labware_file)
            labware_dict[labware_name] = labware_def
    return labware_dict 

In [4]:
def run(protocol, labware_dict):
    protocol.home()

    metadata = {
    'protocolName': 'My Protocol',
    'author': 'Name <email@address.com>',
    'description': 'Simple protocol to get started using OT2',
    'apiLevel': '2.5'
    }

    labware_name = '20mlscintillation_12_wellplate_18000ul'
    
    plate = protocol.load_labware_from_definition(labware_dict[labware_name], 1)
    print(plate)
    # plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '11')
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', '10')

    hold = [j for i in plate.rows() for j in i]
    left_pipette = protocol.load_instrument('p300_single', 'left', tip_racks=[tiprack])

    left_pipette.pick_up_tip()
    left_pipette.aspirate(100, hold[0])
    left_pipette.dispense(100, hold[1])
    left_pipette.drop_tip()



    for line in protocol.commands():
        print(line)

In [10]:
def custom_or_native_labware(protocol, labware_name, labware_slot, custom_labware_dict):
    if labware_name in custom_labware_dict:
        loaded_labware = protocol.load_labware_from_definition(custom_labware_dict[labware_name], labware_slot)
    else: 
        loaded_labware = protocol.load_labware(labware_name, labware_slot)     
    return loaded_labware

In [9]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
labware_dict = custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.0')
test_name_custom = '20mlscintillation_12_wellplate_18000ul'
test_name_native = "corning_96_wellplate_360ul_flat"
test_slot_custom = "1"
test_slot_native = "2"
custom = custom_or_native_labware(protocol, test_name_custom, test_slot_custom, labware_dict)
native = custom_or_native_labware(protocol, test_name_native, test_slot_native, labware_dict)

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


opentrons.protocol_api.labware.Labware